In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import pandas as pd



In [16]:
#load data
data = pd.read_csv('Symptom2Disease.csv')

#split data into train and test
train_data = data.sample(frac=0.8, random_state=0)
test_data = data.drop(train_data.index)

print(len(train_data))
print(f"{len(test_data)}, {test_data.shape})")

for i,x in enumerate(test_data):
    if(i == 10):
        break
    print(x)
    


960
240, (240, 3))


TypeError: string indices must be integers

In [11]:
#connect to GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [13]:
#convert data to tensor
train_data_tensor = torch.tensor(train_data)
test_data_tensor = torch.tensor(test_data)

#split data into input and output
train_input = train_data_tensor[:, 0:2]
train_output = train_data_tensor[:, 2]

test_input = test_data_tensor[:, 0:2]
test_output = test_data_tensor[:, 2]



ValueError: could not determine the shape of object type 'DataFrame'

In [ ]:
#convert data to float
train_input = train_input.float()
train_output = train_output.float()

test_input = test_input.float()
test_output = test_output.float()

#convert data to cuda
train_input = train_input.to(device)
train_output = train_output.to(device)

test_input = test_input.to(device)
test_output = test_output.to(device)



In [ ]:
#define hyperparameters
learning_rate = 0.01
epochs = 1000


In [ ]:
#create model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 10)
        self.fc2 = nn.Linear(10, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.sigmoid(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x

In [ ]:
#define optimizer and loss function
net = Net()
net.to(device)

criterion = nn.BCELoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate)

In [ ]:
#train model
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = net(train_input)
    loss = criterion(outputs, train_output)
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print(f"Epoch {epoch} Loss: {loss.item()}")

In [ ]:
#evaluate model
with torch.no_grad():
    outputs = net(test_input)
    predicted = outputs.round()
    acc = predicted.eq(test_output.view_as(predicted)).sum() / float(test_output.shape[0])
    print(f"Accuracy: {acc.item()}")

